In [3]:
import numpy as np
import pandas as pd

In [4]:
ds=pd.read_csv('marketing_tr.csv')

In [5]:
ds=ds.drop(['marital','schooling','default','housing','loan','campaign','previous','poutcome','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed','pmonths'],axis=1)

In [11]:
ds.head()

,custAge,profession,contact,month,day_of_week,pdays,pastEmail,responded
0,55.0,admin.,0,nov,mon,999,0,no
1,38.0,blue-collar,0,jul,mon,999,0,no
2,42.0,technician,0,may,mon,999,0,no
3,55.0,management,0,jul,wed,999,0,no
4,38.0,admin.,0,may,tue,999,1,no


In [7]:
ds['custAge']=ds['custAge'].fillna(ds['custAge'].median())

In [10]:
ds.loc[ds['contact']=='cellular','contact']=0
ds.loc[ds['contact']=='telephone','contact']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='feb','month']=1
ds.loc[ds['month']=='mar','month']=1
ds.loc[ds['month']=='ap','month']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='jan','month']=1
ds.loc[ds['month']=='jan','month']=1

In [12]:
ds['month'].unique()

array(['nov', 'jul', 'may', 'aug', 'apr', 'jun', 'mar', 'oct', 'sep', 'dec'], dtype=object)

In [ ]:
print ds.shape